# BW \#69 Election participation
Unfamiliar with DuckDB? Simply put, it's an in-memory, in-process database. So when you use it inside of a Python program, it's inside of the same process, with no client, server, or file. It has a complete and rich implementation of SQL, and can run queries on Pandas data frames. A quick tutorial is here: https://duckdb.org/2021/05/14/sql-on-pandas.html

DuckDB is a fast in-process analytical database

## Data and six questions
The data comes from International IDEA's voter turnout database, whose home page is here:

https://www.idea.int/data-tools/data/voter-turnout-database

Go to that page, and click on the "Export data" button. This will download an Excel file. We're interested in the first ("All") sheet in the document.

## Challenges
The learning goals include: Working with DuckDB and SQL, grouping, pivot tables, and working with datetime data.
- Read the Excel file into Pandas as a data frame. However, turn the "Year" column into a datetime column, the numeric columns into float (removing "%" and "," characters), and the "Election type" and "Compulsory voting" columns into categories. How much memory do we save in making such changes?
- Using Pandas, which five countries with non-compulsory voting has the highest mean VAP turnout? Now use DuckDB to calculate the same thing. How long did each query take?


In [2]:
!pip install duckdb

   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB 640.0 kB/s eta 0:00:16
   ---------------------------------------- 0.0/9.9 MB 487.6 kB/s eta 0:00:21
   ---------------------------------------- 0.1/9.9 MB 525.1 kB/s eta 0:00:19
   ---------------------------------------- 0.1/9.9 MB 435.7 kB/s eta 0:00:23
   ---------------------------------------- 0.1/9.9 MB 403.5 kB/s eta 0:00:25
   ---------------------------------------- 0.1/9.9 MB 368.6 kB/s eta 0:00:27
   ---------------------------------------- 0.1/9.9 MB 399.4 kB/s eta 0:00:25
    --------------------------------------- 0.1/9.9 MB 405.9 kB/s eta 0:00:25
    --------------------------------------- 0.2/9.9 MB 436.9 kB/s eta 0:00:23
    --------------------------------------- 0.2/9.9 MB 427.7 kB/s eta 0:00:23
    --------------------------------------- 0.2/9.9 MB 429.5 kB/s eta 0:00:23
    --------------------------------------- 0.2/9.9 MB 430.1 kB/s eta 0:0

In [3]:
import pandas as pd
import duckdb

In [5]:
!pip install openpyxl

  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 435.7 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 393.8 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/250.9 kB 435.7 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.9 kB 476.3 kB/s eta 0:00:01
   ------------------ ------------------- 122.9/250.9 kB 479.3 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/250.9 kB 500.5 kB/s eta 0:00:01
   -------------------------- ----------- 174.1/250.9 kB 499.5 kB/s eta 0:00:01
   --------------------------- ---------- 184.3/250.9 kB 483.6 kB/s eta 0:00:01
   ---------------------------------- --- 225.3/250.9 kB 509.6 kB/s eta 0:00:01
   -------------------------------------- 250.9/250.9 kB 513.2 kB/s et

How much memory does this take up? We can find out by invoking “memory_usage” on our data frame, which returns the number of bytes used by each column. We can then use “sum” to find the total amount of memory used by the data frame.

Note that we need to pass the “deep=True” keyword argument in order to ensure that Python-object columns are calculated correctly:

In [47]:
df = pd.read_excel('idea_export_voter_turnout_database_region.xlsx', sheet_name='All')

In [40]:
df

,Country,ISO2,ISO3,Election Type,Year,Voter Turnout,Total vote,Registration,VAP Turnout,Voting age population,Population,Invalid votes,Compulsory voting
0,Afghanistan,AF,AFG,Parliamentary,2018-10-27,45.23%,"4,000,000","8,843,151",21.81%,"18,340,292","34,940,837",NaN,No
1,Afghanistan,AF,AFG,Parliamentary,2010-09-18,35.14%,"4,216,594","12,000,000",29.71%,"14,191,908","29,120,727",4.42%,No
2,Afghanistan,AF,AFG,Parliamentary,2005-09-18,49.37%,"6,406,615","12,977,336",51.71%,"12,389,532","26,334,702",5.10%,No
3,Afghanistan,AF,AFG,Presidential,2019-09-28,19.00%,"1,824,401","9,600,000",9.63%,"18,938,369","35,780,458",NaN,No
4,Afghanistan,AF,AFG,Presidential,2014-06-14,38.90%,"8,109,493","20,845,988",50.03%,"16,208,255","31,822,848",1.68%,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3693,Zimbabwe,ZW,ZWE,Presidential,2008-06-27,42.37%,"2,514,750","5,934,768",47.27%,"5,320,015","12,311,143",5.23%,No
3694,Zimbabwe,ZW,ZWE,Presidential,2008-03-29,42.75%,"2,537,240","5,934,768",47.69%,"5,320,015","12,311,143",1.58%,No
3695,Zimbabwe,ZW,ZWE,Presidential,2002-01-01,54.33%,"3,046,891","5,607,795",54.25%,"5,615,938","11,365,366",1.60%,No
3696,Zimbabwe,ZW,ZWE,Presidential,1996-01-01,32.30%,"1,557,558","4,822,289",26.68%,"5,839,000","11,678,000",3.00%,No


In [41]:
df.memory_usage(deep=True).sum()

2657758

In [44]:
filename = 'idea_export_voter_turnout_database_region.xlsx'

df = pd.read_excel(filename,
                   sheet_name='All',
                   parse_dates=['Year'])


In [48]:
convert_to_float = ['Voter Turnout', 'Total vote',	'Registration', 'VAP Turnout', 'Voting age population', 'Population', 'Invalid votes']
for one_colname in convert_to_float:
    df[one_colname] = (df
                       [one_colname]
                       .str.replace(r'[,%]', '', regex=True)
                       .astype('float')
                      )


In [49]:
convert_to_category = ['Election Type', 'Compulsory voting']
for one_colname in convert_to_category:
    df[one_colname] = df[one_colname].astype('category')
df.memory_usage(deep=True).sum()


1029151

In [50]:
2657758 - 1029151

1628607

We save 1628607 bytes so 1.6MB

To remove characters, from a column in a DataFrame, you can use the str.replace method.
Pour supprimer des caractères spécifiques comme % et , de toutes les colonnes d'un DataFrame en une seule opération. Vous pouvez utiliser la méthode applymap pour appliquer une fonction à chaque élément du DataFrame.

In [42]:
df['Year'] = pd.to_datetime(df['Year'], errors = 'coerce')

df['Voter Turnout'] = df['Voter Turnout'].str.replace('%', '').str.replace(',', '')
df['Voter Turnout'] = pd.to_numeric(df['Voter Turnout'], errors = 'coerce')
df['Total vote'] = df['Total vote'].str.replace('%', '').str.replace(',', '')
df['Total vote'] = pd.to_numeric(df['Total vote'], errors = 'coerce')
df['Registration'] = df['Registration'].str.replace('%', '').str.replace(',', '')
df['Registration'] = pd.to_numeric(df['Registration'], errors = 'coerce')
df['VAP Turnout'] = df['VAP Turnout'].str.replace('%', '').str.replace(',', '')
df['VAP Turnout'] = pd.to_numeric(df['VAP Turnout'], errors = 'coerce')
df['Voting age population'] = df['Voting age population'].str.replace('%', '').str.replace(',', '')
df['Voting age population'] = pd.to_numeric(df['Voting age population'], errors = 'coerce')
df['Population'] = df['Population'].str.replace('%', '').str.replace(',', '')
df['Population'] = pd.to_numeric(df['Population'], errors = 'coerce')
df['Invalid votes'] = df['Invalid votes'].str.replace('%', '').str.replace(',', '')
df['Invalid votes'] = pd.to_numeric(df['Invalid votes'], errors = 'coerce')
df['Election Type'] = (df['Election Type']).astype('category')
df['Compulsory voting'] = (df['Compulsory voting']).astype('category')
df.memory_usage(deep=True).sum()

840553

In [43]:
2657758 - 840553

1817205

In [52]:
df

,Country,ISO2,ISO3,Election Type,Year,Voter Turnout,Total vote,Registration,VAP Turnout,Voting age population,Population,Invalid votes,Compulsory voting
0,Afghanistan,AF,AFG,Parliamentary,2018-10-27,45.23,4000000.0,8843151.0,21.81,18340292.0,34940837.0,NaN,No
1,Afghanistan,AF,AFG,Parliamentary,2010-09-18,35.14,4216594.0,12000000.0,29.71,14191908.0,29120727.0,4.42,No
2,Afghanistan,AF,AFG,Parliamentary,2005-09-18,49.37,6406615.0,12977336.0,51.71,12389532.0,26334702.0,5.10,No
3,Afghanistan,AF,AFG,Presidential,2019-09-28,19.00,1824401.0,9600000.0,9.63,18938369.0,35780458.0,NaN,No
4,Afghanistan,AF,AFG,Presidential,2014-06-14,38.90,8109493.0,20845988.0,50.03,16208255.0,31822848.0,1.68,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3693,Zimbabwe,ZW,ZWE,Presidential,2008-06-27,42.37,2514750.0,5934768.0,47.27,5320015.0,12311143.0,5.23,No
3694,Zimbabwe,ZW,ZWE,Presidential,2008-03-29,42.75,2537240.0,5934768.0,47.69,5320015.0,12311143.0,1.58,No
3695,Zimbabwe,ZW,ZWE,Presidential,2002-01-01,54.33,3046891.0,5607795.0,54.25,5615938.0,11365366.0,1.60,No
3696,Zimbabwe,ZW,ZWE,Presidential,1996-01-01,32.30,1557558.0,4822289.0,26.68,5839000.0,11678000.0,3.00,No


On economise 1817205 bytes soit 1.8MB

### Using Pandas, which five countries with non-compulsory voting has the highest mean VAP turnout? Now use DuckDB to calculate the same thing. How long did each query take?


In [61]:
%%timeit
df[df['Compulsory voting'] == 'No'].groupby('Country')['VAP Turnout'].mean().sort_values(ascending=False).head(5)

1.04 ms ± 45.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Croatia                         303.760769

North Macedonia, Republic of    293.735217

Somalia                         129.466667

Rwanda                          101.034444

Cook Islands                    100.640000

Um, wait a second. How can there be 315 percent voter turnout? We might be aiming for high turnout, but that seems a bit … high, no? The FAQ at https://www.idea.int/data-tools/data/voter-turnout-database indicates that VAP (“voting age population”), along with turnout and other numbers, are often estimates, and that they aren’t always updated in sync. That said, we can probably assume that voter turnout in Croatia and Maceconia is quite high, even if not quite 315 percent.

What about the next three? Well, Somalia isn’t exactly a bastion of open democracy. The Cook Islands have a population of 15,000 people, so it seems likely to me that they have high turnout rates. And Vietnam … it’s a one-party state, so I’m not sure who people are voting for, but it’s good to know (I guess) that they’re voting in high percentages.

Bottom line, having a very high percentage of the public coming to vote doesn’t necessarily mean that you’re a model democracy.





#### With DuckDB

DuckDB is, as I wrote above, an in-memory relational database that uses SQL. It uses standard SQL queries to create tables, update them, and retrieve from them. However, the tables all reside in memory. Moreover, it’s a columnar database, meaning that (like Pandas) it is structured primarily along columns, rather than along rows, as traditional databases did. It’s not meant for high-speed transactions; rather, DuckDB is designed for high-performance, in-process analysis of data.

In [62]:
duckdb.query('''SELECT Country, mean("VAP Turnout") as mvt
                FROM df
                WHERE "Compulsory voting" = 'No'
                GROUP BY Country
                ORDER BY mvt DESC
                LIMIT 5''')


┌──────────────────────────────┬────────────────────┐
│           Country            │        mvt         │
│           varchar            │       double       │
├──────────────────────────────┼────────────────────┤
│ Croatia                      │  303.7607692307693 │
│ North Macedonia, Republic of │  293.7352173913044 │
│ Somalia                      │ 129.46666666666667 │
│ Rwanda                       │ 101.03444444444445 │
│ Cook Islands                 │             100.64 │
└──────────────────────────────┴────────────────────┘